# Scrape all articles

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pickle
import newspaper
from newspaper import Article
import pandas as pd
import pickle

In [2]:
# Import big CSV
bigdf = pd.DataFrame.from_csv("idmc_uniteideas_input_url.csv")
bigdf.shape

(599624, 2)

In [3]:
# Rename columns
bigdf = bigdf.rename(columns = {"DATE" : "date"})
bigdf = bigdf.rename(columns = {"DocumentIdentifier" : "url"})

In [4]:
bigdf.head()

,date,url
GKGRECORDID,,
20160512000000-2927,20160512000000,http://www.somalilandpress.com/iom-supports-dr...
20161114080000-854,20161114080000,http://www.9and10news.com/story/33699640/syria...
20161219200000-704,20161219200000,http://tribune.com.pk/story/1268397/saudi-led-...
20161219200000-177,20161219200000,http://www.tv360nigeria.com/nigerias-humanitar...
20161219200000-900,20161219200000,http://www.couriermail.com.au/news/world/ecuad...


In [5]:
# Collect all urls
all_filter_urls = []

for url in bigdf.url:
    all_filter_urls.append(url)
    
len(all_filter_urls)

599624

## Scrape article by article, store it in MongoDB

In [6]:
# Initiate Mongo
from pymongo import MongoClient
client = MongoClient('ec2-##-##-###-##.compute-1.amazonaws.com', 27017)

In [10]:
# AWS collection
aws_0 = client.idetect.aws_0

In [11]:
def html_pages(url):
    info = {}
    
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    valid = article.is_valid_url()
    authors = article.authors
    date = article.publish_date
    text = article.text
    keywords = article.keywords
    summary = article.summary
    image = article.top_image

    info['url'] = url
    info['valid'] = valid
    info['authors'] = authors
    info['date'] = date
    info['text'] = text
    info['keywords'] = keywords
    info['summary'] = summary
    info['image'] = image

    aws_0.insert_one(info)

In [ ]:
for url in all_filter_urls:
    try:
        html_pages(url)    
    except:
        continue 